In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
import openai

In [4]:
from langchain.chat_models import ChatOpenAI

In [5]:
from langchain.prompts import ChatPromptTemplate

# 5. Load cleaned data

In [6]:
df = pd.read_csv("01. Cleaned_retail_data.csv")

In [7]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,profit_margin,Cost_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,0.193525,5.605004
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,0.475850,3.538012
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,0.368677,4.261430
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,0.303343,1.462980
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,0.086449,1.141939


# 6. Set Openai API

In [9]:
# llm_model = "gpt-4-1106-preview"
llm_model = "gpt-3.5-turbo-0125"

In [10]:
chat = ChatOpenAI(temperature=0.0, model=llm_model)

In [11]:
review_template = """\
For the following text, choose only one category which is the best from the list
and do not create new category apart from this list.

1. Clothing and Accessories
2. Electronics
3. Food and Beverages
4. Health and Beauty
5. Home and Garden
6. Toys and Games
7. Sports and Outdoors
8. Books and Stationery
9. Automotive
10. Uncategorized

For example:

Question: SET OF 6 SPICE TINS PANTRY DESIGN
Answer: Home and Garden

Question: POSTCARD
Answer: Books and Stationery

Question: ??COUPON??
Answer: Uncategorized

Question: {text}
Answer:
"""

In [12]:
prompt_template = ChatPromptTemplate.from_template(review_template)

In [13]:
item_list = []
cat_list = []

In [14]:
len(df['Description'].unique().tolist())

4595

In [15]:
count = 1

In [16]:
for item_input in df['Description'].unique().tolist():
    try:
        messages = prompt_template.format_messages(text=item_input)
        chat = ChatOpenAI(temperature=0.0, model=llm_model)
        response = chat(messages)

        cat = response.content
        cat = cat.replace("Answer", "")
        cat = cat.replace("answer", "")
        cat = cat.replace("Category", "")
        cat = cat.replace("category", "")
        cat = cat.replace(":", "")
        cat = cat.strip()
        
        cat_list.append(response.content)
        item_list.append(item_input)
        print(count, item_input, ":",response.content)
        count+= 1
    except:
        break

1 15CM CHRISTMAS GLASS BALL 20 LIGHTS : Home and Garden
2 PINK CHERRY LIGHTS : Home and Garden
3 WHITE CHERRY LIGHTS : Home and Garden
4 RECORD FRAME 7" SINGLE SIZE : Home and Garden
5 STRAWBERRY CERAMIC TRINKET BOX : Home and Garden
6 PINK DOUGHNUT TRINKET POT : Home and Garden
7 SAVE THE PLANET MUG : Home and Garden
8 FANCY FONT HOME SWEET HOME DOORMAT : Home and Garden
9 CAT BOWL : Home and Garden
10 DOG BOWLCHASING BALL DESIGN : Toys and Games
11 HEART MEASURING SPOONS LARGE : Home and Garden
12 LUNCHBOX WITH CUTLERY FAIRY CAKES : Home and Garden
13 DOOR MAT BLACK FLOCK : Home and Garden
14 LOVE BUILDING BLOCK WORD : Toys and Games
15 HOME BUILDING BLOCK WORD : Home and Garden
16 ASSORTED COLOUR BIRD ORNAMENT : Home and Garden
17 PEACE WOODEN BLOCK LETTERS : Home and Garden
18 CHRISTMAS CRAFT WHITE FAIRY : Toys and Games
19 HEART IVORY TRELLIS LARGE : Home and Garden
20 HEART FILIGREE DOVE LARGE : Home and Garden
21 FULL ENGLISH BREAKFAST PLATE : Food and Beverages
22 PIZZA PLATE I

In [ ]:
set(cat_list)

In [ ]:
***

# 7. Append to dataframe

In [ ]:
df.head()

In [ ]:
len(cat_list)

In [ ]:
len(item_list)

In [ ]:
df_cat = pd.DataFrame({
    'Category': cat_list,
    'Description': item_list
})

In [ ]:
df_cat.head()

In [ ]:
df = pd.merge(df, df_cat, on='Description',how='inner')

In [ ]:
df = df[df['Category'] != 'Uncategorized']

# 8. Export cleaned data

In [ ]:
df.head()

In [ ]:
df.to_csv('02. Labeled_retail_data.csv', sep=',', index=False)